# **The fourth in-class-exercise (40 points in total, 03/29/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [2]:
# Write your code here
import nltk
nltk.download('stopwords')

import re
import numpy as np
import pandas as pd
from pprint import pprint




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
!pip install pyLDAvis


     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=3e1a6f3e479af5a4c0084312c68da2d14bc5aa6a09a3da0d1b4a8d95c1b12d55
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [4]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models 
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
# Import Dataset
df = pd.read_csv('/content/sample_data/train.csv')
df.head(100)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
95,95,White House Confirms More Gitmo Transfers Befo...,Edwin Mora,President Barack Obama will likely release mor...,0
96,96,The Geometry of Energy and Meditation of Buddha,NaN,License DMCA \nA mandala is a visual symbol of...,1
97,97,Poll: Most Voters Have Not Heard of Democratic...,Katherine Rodriguez,There is a minefield of potential 2020 electio...,0
98,98,Migrants Confront Judgment Day Over Old Deport...,Vivian Yee,There are a little more than two weeks between...,0


In [8]:
# Convert to list
data = df.text.values.tolist()






pprint(data[:1])

['House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz '
 'Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on '
 'the stump in American Fork, Utah ( image courtesy Michael Jolley, available '
 'under a Creative Commons-BY license) \n'
 'With apologies to Keith Olbermann, there is no doubt who the Worst Person in '
 'The World is this week–FBI Director James Comey. But according to a House '
 'Democratic aide, it looks like we also know who the second-worst person is '
 'as well. It turns out that when Comey sent his now-infamous letter '
 'announcing that the FBI was looking into emails that may be related to '
 'Hillary Clinton’s email server, the ranking Democrats on the relevant '
 'committees didn’t hear about it from Comey. They found out via a tweet from '
 'one of the Republican committee chairmen. \n'
 'As we now know, Comey notified the Republican chairmen and Democratic '
 'ranking members of the House Intelligence, Judiciary, and Ov

In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['house', 'dem', 'aide', 'we', 'didn', 'even', 'see', 'comey', 'letter', 'until', 'jason', 'chaffetz', 'tweeted', 'it', 'by', 'darrell', 'lucus', 'on', 'october', 'subscribe', 'jason', 'chaffetz', 'on', 'the', 'stump', 'in', 'american', 'fork', 'utah', 'image', 'courtesy', 'michael', 'jolley', 'available', 'under', 'creative', 'commons', 'by', 'license', 'with', 'apologies', 'to', 'keith', 'olbermann', 'there', 'is', 'no', 'doubt', 'who', 'the', 'worst', 'person', 'in', 'the', 'world', 'is', 'this', 'week', 'fbi', 'director', 'james', 'comey', 'but', 'according', 'to', 'house', 'democratic', 'aide', 'it', 'looks', 'like', 'we', 'also', 'know', 'who', 'the', 'second', 'worst', 'person', 'is', 'as', 'well', 'it', 'turns', 'out', 'that', 'when', 'comey', 'sent', 'his', 'now', 'infamous', 'letter', 'announcing', 'that', 'the', 'fbi', 'was', 'looking', 'into', 'emails', 'that', 'may', 'be', 'related', 'to', 'hillary', 'clinton', 'email', 'server', 'the', 'ranking', 'democrats', 'on', 'the'

In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['house', 'dem', 'aide', 'we', 'didn', 'even', 'see', 'comey', 'letter', 'until', 'jason_chaffetz', 'tweeted', 'it', 'by', 'darrell_lucus', 'on', 'october', 'subscribe', 'jason_chaffetz', 'on', 'the', 'stump', 'in', 'american', 'fork', 'utah', 'image_courtesy', 'michael', 'jolley', 'available', 'under_creative_commons', 'by', 'license', 'with', 'apologies', 'to', 'keith_olbermann', 'there', 'is', 'no', 'doubt', 'who', 'the', 'worst', 'person', 'in', 'the', 'world', 'is', 'this', 'week', 'fbi_director_james', 'comey', 'but', 'according', 'to', 'house', 'democratic', 'aide', 'it', 'looks', 'like', 'we', 'also', 'know', 'who', 'the', 'second', 'worst', 'person', 'is', 'as', 'well', 'it', 'turns', 'out', 'that', 'when', 'comey', 'sent', 'his', 'now', 'infamous', 'letter', 'announcing', 'that', 'the', 'fbi', 'was', 'looking', 'into', 'emails', 'that', 'may', 'be', 'related', 'to', 'hillary', 'clinton', 'email_server', 'the', 'ranking', 'democrats', 'on', 'the', 'relevant', 'committees', 'di

In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['aide', 'even', 'see', 'tweet', 'available', 'license', 'apology', 'doubt', 'bad', 'person', 'world', 'week', 'accord', 'democratic', 'aide', 'look', 'also', 'know', 'second', 'bad', 'person', 'well', 'turn', 'send', 'infamous', 'letter', 'announce', 'look', 'email', 'may', 'related', 'rank', 'relevant', 'committee', 'hear', 'find', 'republican', 'know', 'notify', 'republican', 'chairman', 'democratic', 'rank', 'review', 'email', 'recently', 'discover', 'order', 'see', 'contained_classified', 'information', 'long', 'letter', 'go', 'chairman', 'set', 'political', 'world', 'fbi_dir', 'inform', 'learn', 'existence', 'email', 'appear_pertinent', 'investigation', 'case', 'reopen', 'course', 'know', 'case', 'actually', 'say', 'review', 'email', 'light', 'know', 'sexte', 'teenager', 'apparently', 'little', 'thing', 'fact', 'matter', 'already', 'vow', 'initiate', 'raft', 'investigation', 'win', 'least', 'year', 'worth', 'possibly', 'entire', 'term', 'worth', 'apparently', 'chaffetz', 'think'

In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 3), (2, 1), (3, 1), (4, 1), (5, 4), (6, 3), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 3), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 3), (23, 7), (24, 4), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 6), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 4), (50, 1), (51, 4), (52, 1), (53, 1), (54, 2), (55, 2), (56, 1), (57, 2), (58, 1), (59, 3), (60, 1), (61, 1), (62, 1), (63, 3), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 3), (80, 1), (81, 8), (82, 1), (83, 1), (84, 2), (85, 2), (86, 4), (87, 1), (88, 2), (89, 1), (90, 1), (91, 1), (92, 3), (93, 1), (94, 1), (95, 4), (96, 1), (97, 1), (98, 4), (99, 1), (100, 3), (101, 1), (102, 1), (103, 1), (104, 1), (105, 2), (106, 1), (107, 1), (108, 1), (109, 3), (110, 1)

In [14]:
id2word[0]

'abusive'

In [15]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('abusive', 1),
  ('accord', 3),
  ('act', 1),
  ('actually', 1),
  ('advance', 1),
  ('aide', 4),
  ('already', 3),
  ('also', 2),
  ('anchor', 1),
  ('announce', 1),
  ('answer', 1),
  ('apology', 1),
  ('apparently', 2),
  ('appear_pertinent', 1),
  ('attempt', 1),
  ('available', 1),
  ('bad', 3),
  ('basic', 1),
  ('become', 1),
  ('bipartisanship', 1),
  ('boss', 1),
  ('briefly', 1),
  ('case', 3),
  ('chaffetz', 7),
  ('chairman', 4),
  ('check', 1),
  ('colleague', 1),
  ('college', 1),
  ('comey', 1),
  ('committee', 2),
  ('connect', 1),
  ('consider', 1),
  ('contained_classified', 1),
  ('control', 1),
  ('cook', 1),
  ('cooler_head', 1),
  ('counterpart', 1),
  ('course', 1),
  ('courtesy', 1),
  ('darrell', 1),
  ('decency', 1),
  ('democrat', 1),
  ('democratic', 6),
  ('desire', 1),
  ('development', 1),
  ('director', 1),
  ('discover', 1),
  ('district', 1),
  ('doubt', 1),
  ('email', 4),
  ('entire', 1),
  ('even', 4),
  ('existence', 1),
  ('expedition', 1),
  (

In [16]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [17]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.026*"food" + 0.019*"health" + 0.016*"doctor" + 0.016*"patient" + '
  '0.014*"eat" + 0.012*"medical" + 0.009*"help" + 0.009*"also" + '
  '0.009*"cancer" + 0.009*"treatment"'),
 (1,
  '0.011*"artist" + 0.011*"wear" + 0.010*"room" + 0.009*"restaurant" + '
  '0.009*"dinner" + 0.009*"art" + 0.008*"space" + 0.008*"hotel" + '
  '0.008*"guest" + 0.008*"dog"'),
 (2,
  '0.026*"gold" + 0.022*"die" + 0.010*"king" + 0.009*"pasaporte" + '
  '0.008*"silver" + 0.007*"click" + 0.007*"http_www" + 0.006*"com" + '
  '0.005*"subscribe" + 0.005*"songwriter"'),
 (3,
  '0.128*"say" + 0.019*"year" + 0.015*"last" + 0.012*"week" + 0.012*"time" + '
  '0.010*"also" + 0.010*"tell" + 0.010*"take" + 0.010*"make" + 0.010*"call"'),
 (4,
  '0.023*"city" + 0.013*"people" + 0.013*"year" + 0.010*"many" + 0.010*"area" '
  '+ 0.010*"leave" + 0.010*"local" + 0.010*"fire" + 0.008*"live" + '
  '0.007*"large"'),
 (5,
  '0.039*"company" + 0.017*"business" + 0.014*"money" + 0.010*"sell" + '
  '0.010*"industry" + 0.009*"p

In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.16905680750388

Coherence Score:  0.4771990713729447


In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.054987 -0.188437       1        1  14.776712
3      0.093798 -0.127928       2        1  11.177878
14     0.204551  0.027083       3        1   8.291182
8      0.105149 -0.049922       4        1   7.270353
13    -0.117876 -0.055039       5        1   5.834727
9      0.183345  0.120434       6        1   5.546600
4     -0.013153 -0.035488       7        1   4.915781
18     0.075055 -0.146694       8        1   4.656483
15     0.095376  0.047829       9        1   4.625906
6      0.140782  0.121555      10        1   4.484477
17     0.033977  0.116816      11        1   4.358950
5      0.056730 -0.082052      12        1   4.278084
19    -0.098630 -0.104542      13        1   3.716715
7      0.057358  0.116303      14        1   3.322915
10    -0.111957 -0.094065      15        1   2.965797
12     0.020955  0.216057      16        1   2.902476
16    -0.029729 -0.006425      17        1   2.629690
0     -0.161224 -0.073793      18        1   2.020059
1     -0.285376 -0.040737      19        1   1.723796
2     -0.304117  0.239047      20        1   0.501417, topic_info=           Term          Freq         Total Category  logprob  loglift
134         say  97074.000000  97074.000000  Default  30.0000  30.0000
340       trump  20953.000000  20953.000000  Default  29.0000  29.0000
233    election  13513.000000  13513.000000  Default  28.0000  28.0000
296      people  35531.000000  35531.000000  Default  27.0000  27.0000
64           go  29574.000000  29574.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
76497   спасибо     78.595343     83.939149  Topic20  -5.9430   5.2297
7447   http_www    196.679299    237.128821  Topic20  -5.0257   5.1085
646         die    663.190438   4100.366835  Topic20  -3.8102   3.4737
3498      click    200.260335    733.022004  Topic20  -5.0077   3.9979
3834        com    172.172406   3140.785608  Topic20  -5.1588   2.3918

[1344 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
2648       3  0.998956  abortion
6242       4  0.998013    absurd
1643      13  0.998659  accident
1          2  0.142691    accord
1          3  0.043969    accord
...      ...       ...       ...
76497     20  0.941158   спасибо
22779     20  0.994520       что
14555     17  0.995111     чтобы
4513      20  0.987983      этих
12519     20  0.988515      этом

[3404 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 4, 15, 9, 14, 10, 5, 19, 16, 7, 18, 6, 20, 8, 11, 13, 17, 1, 2, 3])

In [20]:

!wget "http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip"
!unzip "/content/mallet-2.0.8.zip"

--2022-04-05 03:38:31--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip [following]
--2022-04-05 03:38:31--  https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  11.2MB/s    in 1.4s    

2022-04-05 03:38:33 (11.2 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]

Archive:  /content/mallet-2.0.8.zip
   creating: mallet-2.0.8/
   creating: mallet-2.0.8/bin/
  inflating: mallet-2.0.8/bin/classifier2info  
  inflating: mallet-2.0.8/bin/csv2classify  
  inflating: 

In [24]:
mallet_path = '/content/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [25]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nThe Measure of coherence: ', coherence_ldamallet)

[(8,
  [('company', 0.03722692043483154),
   ('business', 0.01828745299092225),
   ('year', 0.01760439023065011),
   ('work', 0.015819112561757023),
   ('pay', 0.014181314352468147),
   ('plan', 0.01236498837628996),
   ('make', 0.011491754733896601),
   ('service', 0.010304156980241633),
   ('industry', 0.009624975258380132),
   ('sell', 0.009473614760365283)]),
 (18,
  [('family', 0.01893257495155607),
   ('day', 0.014818380779391002),
   ('make', 0.01380297985784387),
   ('child', 0.011027820367081591),
   ('time', 0.010623278167262018),
   ('friend', 0.010024555711529048),
   ('home', 0.009458196631781645),
   ('find', 0.00901724563397831),
   ('mother', 0.008612703434158734),
   ('room', 0.00643222097713123)]),
 (19,
  [('news', 0.028787487765804632),
   ('medium', 0.025159835806142176),
   ('post', 0.02426387881207789),
   ('story', 0.017524724030637834),
   ('comment', 0.016672591019978866),
   ('twitter', 0.016151572550604527),
   ('article', 0.015693855390593425),
   ('report'

In [26]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# LSA Model
number_of_topics=7
words=10
document_list,titles=load_data("","/content/sample_data/sample1_exercise.txt")
clean_text=preprocess_data(document_list)
model=create_gensim_lsa_model(clean_text,number_of_topics,words)

In [ ]:
model=create_gensim_lsa_model(preprocess_data(data),number_of_topics,words)

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
# Write your code here
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt





In [ ]:
#import modules
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt


In [ ]:
def load_data(path,file_name):
    """
    Input  : path and file_name
    Purpose: loading text file
    Output : list of paragraphs/documents and
             title(initial 100 words considred as title of document)
    """
    documents_list = []
    titles=[]
    with open( os.path.join(path, file_name) ,"r") as fin:
        for line in fin.readlines():
            text = line.strip()
            documents_list.append(text)
    print("Total Number of Documents:",len(documents_list))
    titles.append( text[0:min(len(text),100)] )
    return documents_list,titles

In [ ]:
def preprocess_data(doc_set):
    """
    Input  : docuemnt list
    Purpose: preprocess text (tokenize, removing stopwords, and stemming)
    Output : preprocessed text
    """
    # initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    # create English stop words list
    en_stop = set(stopwords.words('english'))
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    # list for tokenized documents in loop
    texts = []
    # loop through document list
    for i in doc_set:
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        # add tokens to list
        texts.append(stemmed_tokens)
    return texts


In [ ]:
def prepare_corpus(doc_clean):
    """
    Input  : clean document
    Purpose: create term dictionary of our courpus and Converting list of documents (corpus) into Document Term Matrix
    Output : term dictionary and Document Term Matrix
    """
    # Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
    dictionary = corpora.Dictionary(doc_clean)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    # generate LDA model
    return dictionary,doc_term_matrix


In [ ]:
def create_gensim_lsa_model(doc_clean,number_of_topics,words):
    """
    Input  : clean document, number of topics and number of words associated with each topic
    Purpose: create LSA model using gensim
    Output : return LSA model
    """
    dictionary,doc_term_matrix=prepare_corpus(doc_clean)
    # generate LSA model
    lsamodel = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
    print(lsamodel.print_topics(num_topics=number_of_topics, num_words=words))
    return lsamodel


In [ ]:
def compute_coherence_values(dictionary, doc_term_matrix, doc_clean, stop, start=2, step=3):
    """
    Input   : dictionary : Gensim dictionary
              corpus : Gensim corpus
              texts : List of input texts
              stop : Max num of topics
    purpose : Compute c_v coherence for various number of topics
    Output  : model_list : List of LSA topic models
              coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, stop, step):
        # generate LSA model
        model = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values


In [ ]:
def plot_graph(doc_clean,start, stop, step):
    dictionary,doc_term_matrix=prepare_corpus(doc_clean)
    model_list, coherence_values = compute_coherence_values(dictionary, doc_term_matrix,doc_clean,
                                                            stop, start, step)
    # Show graph
    x = range(start, stop, step)
    plt.plot(x, coherence_values)
    plt.xlabel("Number of Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')
    plt.show()

In [ ]:
start,stop,step=2,12,1
plot_graph(clean_text,start,stop,step)

NameError: ignored

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 2))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,2), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)
                                                  
                                                  
                                                  
                                                  

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords','Text']

# Show
df_dominant_topic.head(100)

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords","Text"]

# Show
sent_topics_sorteddf_mallet.head()

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [ ]:
import pyLDAvis
pyLDAvis.enable_notebook()

In [ ]:
npz = np.load(open('topics.pyldaviszx .npz', 'r'))
dat = {k: v for (k, v) in npz.iteritems()}
dat['vocab'] = dat['vocab'].tolist()
# dat['term_frequency'] = dat['term_frequency'] * 1.0 / dat['term_frequency'].sum()

FileNotFoundError: ignored

## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [ ]:
# Write your code here





## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

In [ ]:
# Write your answer here (no code needed for this question)
"""Topic Modeling can be assessed by the idea of Topic Coherence, as theme models gives no certification on the interpretability
of their output.Topic displaying furnishes us with strategies to coordinate, comprehend and sum up huge assortments of text based data.
There are numerous procedures that are utilized to get point models.

Two among them are Latent Semantic Analysis(LSA) and Latent Dirichlet Allocation (LDA) .

For LDA:

LDA is a broadly utilized point demonstrating strategy to extricate subject from the literary information.
Subject models learn themes commonly addressed as sets of significant words-consequently from unlabelled archives in a solo manner.
This is an alluring strategy to carry construction to in any case unstructured text information, yet Topics are not destined to be well interpretable, thusly, soundness measures have been proposed to recognize great and awful subjects.
We made word reference and corpus expected for Topic Modeling: The two primary contributions to the LDA subject model are the word reference and the corpus. Gensim makes a one of a kind id for each word in the record.
Notwithstanding the corpus and word reference, we really want to give the quantity of subjects also.
LDA is a solo procedure, implying that we don't be aware before running the model the number of subjects exits in our corpus.We can utilize LDA perception device pyLDAvis, by attempting a couple of quantities of points and contrasting the outcomes.
Subject soundness is one of the primary strategies used to assess the quantity of points.
We want to construct numerous LDA models with various upsides of the quantity of points (k) and pick the one that gives the most elevated soundness esteem. Picking a 'k' that denotes the finish of a quick development of theme intelligence generally offers significant and interpretable subjects.

The plot of soundness esteem Vs number of subjects shows that rationality score diminishes with the quantity of points, with a downfall between 15 to 20.Now, picking the quantity of themes actually relies upon our prerequisite since theme around 10 have great intelligibility scores yet may have rehashed catchphrases in the theme.
and furthermore intelligibility esteem noticed for the information it is 0.76 to utilize LDA calculation

For LSA:

LSA (Latent Semantic Analysis) otherwise called LSI (Latent Semantic Index). LSA utilizes pack of word(BoW) model, which brings about a term-record matrix(occurrence of terms in an archive).
LSA learns dormant subjects by playing out a grid disintegration on the report term lattice utilizing Singular worth deterioration. LSA is commonly utilized as an aspect decrease or sound diminishing method.
The most effective way to decide k (number of subjects) in theme displaying, is by utilizing Topic rationality measure which is a practical measure for distinguishing the quantity of points.
It utilizes the inactive variable models. Each produced theme has a rundown of words. In point lucidness measure, we will view as normal/middle of pairwise word likeness scores of the words in a theme. The high worth of theme cognizance score model will be considered as a decent point model.

The plot of rationality esteem Vs number of points shows that intelligence score diminishes with the quantity of subjects. Of the quantity of themes, 7 have the most noteworthy intelligence score, so the ideal number of subjects are 7.
and furthermore soundness esteem noticed for the information it is 0.54 to utilize LDA calculation

So from the above examination ,I could express that for the information I have taken LDA(Latent Dirichlet Allocation)gives better outcomes with an ideal subject of 10 an


